In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from keras.layers import Average, Input
import tensorflow as tf
import numpy as np
from glob import glob
import mne
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
model_paths = ["../model/client_model{}.h5".format(i) for i in range(10)]

In [3]:
models = [load_model(model_path) for model_path in model_paths]

In [4]:
all_files_path=glob('../test/*.edf')
print(len(all_files_path))
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]

print(len(healthy_file_path),len(patient_file_path))
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']  
      
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)
    return data #trials,channel,length

18
9 9


In [5]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [6]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
data_array=np.moveaxis(data_array,1,2)

9 9
18 18


In [7]:
accuracies =[0.47329649329185486, 0.47329649329185486, 0.47329649329185486, 0.47329649329185486, 0.7753222584724426, 0.5267034769058228, 0.5267034769058228, 0.5267034769058228, 0.5267034769058228, 0.5267034769058228]

In [8]:
weights = [accuracy / sum(accuracies) for accuracy in accuracies]

In [9]:
def weighted_arithmetic_mean(predictions, weights):
    """
    Calculate the weighted arithmetic mean of predictions from multiple models.

    Parameters:
    - predictions: List of numpy arrays representing the predictions from each model.
    - weights: List of weights corresponding to each model.

    Returns:
    - Weighted arithmetic mean of the predictions.
    """
    if len(predictions) != len(weights):
        raise ValueError("Number of models and weights should be the same.")

    weighted_sum = np.zeros_like(predictions[0])

    for i in range(len(predictions)):
        weighted_sum += weights[i] * predictions[i]

    total_weight = np.sum(weights)
    weighted_mean = weighted_sum / total_weight

    return weighted_mean

In [10]:
predictions_list = [model.predict(data_array) for model in models]


# Use the provided function to calculate the weighted mean
weighted_mean_result = weighted_arithmetic_mean(predictions_list, weights)


17/17 [==============================] - 1s 18ms/step


In [11]:
weighted_mean_result

array([[0.5706255 ],
       [0.63828236],
       [0.6418412 ],
       [0.49672395],
       [0.49734336],
       [0.6008173 ],
       [0.61742586],
       [0.60465974],
       [0.62144375],
       [0.5865076 ],
       [0.5233339 ],
       [0.5810405 ],
       [0.5134751 ],
       [0.4994892 ],
       [0.5063811 ],
       [0.6302821 ],
       [0.61743844],
       [0.54612315],
       [0.6271522 ],
       [0.5274973 ],
       [0.52920187],
       [0.64209974],
       [0.6428549 ],
       [0.64280874],
       [0.64277893],
       [0.64281106],
       [0.64287966],
       [0.6427267 ],
       [0.642744  ],
       [0.6428528 ],
       [0.6429093 ],
       [0.64293474],
       [0.6429193 ],
       [0.6429107 ],
       [0.64291483],
       [0.64293283],
       [0.6429275 ],
       [0.6428886 ],
       [0.64293253],
       [0.64292634],
       [0.64291906],
       [0.6429764 ],
       [0.49670604],
       [0.496707  ],
       [0.49670678],
       [0.49670815],
       [0.49670637],
       [0.496

In [12]:
from sklearn.metrics import accuracy_score

# Assuming 'weighted_mean_result' is the result obtained from the weighted arithmetic mean
binary_predictions = (weighted_mean_result > 0.5).astype(int)
# Calculate accuracy
accuracy = accuracy_score(label_array, binary_predictions)

print("Accuracy:", accuracy)

Accuracy: 0.7569060773480663
